In [3]:
from database.adatabase import ADatabase
from extractor.alp_client_extractor import ALPClientExtractor
from extractor.tiingo_extractor import TiingoExtractor
from processor.processor import Processor as processor
from datetime import datetime, timedelta
from tqdm import tqdm 
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
from time import sleep
import matplotlib.pyplot as plt



In [31]:
extractor = ALPClientExtractor(os.getenv("APCAKEY"),os.getenv("APCASECRET"))
tickers = ["AAVE", "AVAX", "BAT", "BCH", "BTC", "CRV", "DOGE", "DOT", "ETH", "GRT", "LINK", "LTC", "MKR", "SHIB", "SUSHI","USDT","USDC", "UNI", "XTZ", "YFI"]
tickers = [x +"/USD" for x in tickers]
prices = []
for ticker in tqdm(tickers):
    price_df = []
    for i in range(1,52):
        start = datetime.now() - timedelta(days=7*i)
        end = datetime.now() - timedelta(days=7*(i-1))
        response = extractor.crypto(ticker,start,end)
        price = pd.DataFrame(response["bars"][ticker]).rename(columns={"c":"adjclose","t":"date"})[["date","adjclose"]]
        price["date"] = pd.to_datetime(price["date"])
        price = price.sort_values("date")
        price_df.append(price)
    price = pd.concat(price_df).sort_values("date")
    price["rolling_return"] = ((price["adjclose"].rolling(30).mean() - price["adjclose"]) / price["adjclose"])
    price["return"] = price["adjclose"].pct_change(5)
    price["avg_return"] = price["return"].rolling(30).mean()
    price["coev"] = (price["adjclose"].rolling(30).mean() / price["adjclose"].rolling(30).std())
    price["actual_return"] = (price["adjclose"].shift(-5) - price["adjclose"]) / price["adjclose"]
    price["ticker"]= ticker
    prices.append(price)

prices = pd.concat(prices).sort_values("date").dropna()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [04:08<00:00, 12.41s/it]


In [28]:
metrics = [
    "rolling_return",
    "return",
    "avg_return"
    ,"coev"
]

In [36]:
date_range = list(prices["date"].unique())

cash = 100
base_position = {"date":date_range[0],"ticker":"","quantity":0,"adjclose":0,"buy_price":0,"buy_date":date_range[0]}
positions = []
analysis = []
for metric in tqdm(metrics):
    for ascending in [True,False]:
        trades = prices.sort_values(metric,ascending=False).groupby("date").first().reset_index().sort_values("date")
        trades = trades[::5]
        trades["actual_return"] = trades["actual_return"] + 1
        trades["cum_return"] = trades["actual_return"].cumprod()
        trades["metric"] = metric
        trades["ascending"] = ascending
        analysis.append(trades.iloc[-1])
        # position = base_position.copy()
        # for date in date_range:
        #     try:
        #         position = position.copy()
        #         position["date"] = date
        #         today = prices[prices["date"]==date]
        #         opportunity = today.sort_values(metric,ascending=ascending).iloc[0]
        #         if today.index.size >= len(tickers):
        #             if position["ticker"] != "":
        #                 position["adjclose"] = today[today["ticker"]==position["ticker"]]["adjclose"].iloc[0].item()
        #                 if (date - position["buy_date"]).days > float(5/(24*60)):
        #                     cash = position["adjclose"] * position["quantity"]
        #                     position = base_position.copy()
        #                     position["date"] = date
        #                     position["ticker"] = opportunity["ticker"]
        #                     position["quantity"] = cash * 0.9975 / opportunity["adjclose"] 
        #                     position["adjclose"] = opportunity["adjclose"]
        #                     position["buy_price"] = opportunity["adjclose"]
        #                     position["buy_date"] = date
        #             else:
        #                     position["date"] = date
        #                     position["ticker"] = opportunity["ticker"]
        #                     position["quantity"] = cash * 0.9975 / opportunity["adjclose"] 
        #                     position["adjclose"] = opportunity["adjclose"]
        #                     position["buy_price"] = opportunity["adjclose"]
        #                     position["buy_date"] = date
        #         positions.append(position.copy())
        #     except Exception as e:
        #         print(str(e))
        # position["metric"] = metric
        # position["ascending"] = ascending
        # position["pv"] = position["adjclose"] * position["quantity"]
        # analysis.append(position)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:31<00:00,  7.86s/it]


In [37]:
a = pd.DataFrame(analysis)
a.sort_values("cum_return",ascending=False)

,date,adjclose,rolling_return,return,avg_return,coev,actual_return,ticker,cum_return,metric,ascending
513680,2024-09-25 23:51:00+00:00,4.620995,0.004313,-0.002516,-0.001023,464.830659,1.003085,DOT/USD,8.880516e+09,rolling_return,True
513680,2024-09-25 23:51:00+00:00,4.620995,0.004313,-0.002516,-0.001023,464.830659,1.003085,DOT/USD,8.880516e+09,rolling_return,False
513680,2024-09-25 23:51:00+00:00,0.999937,0.000042,-0.000182,0.000011,9471.527448,0.999978,USDC/USD,1.005906e+00,coev,True
513680,2024-09-25 23:51:00+00:00,0.999937,0.000042,-0.000182,0.000011,9471.527448,0.999978,USDC/USD,1.005906e+00,coev,False
513680,2024-09-25 23:51:00+00:00,0.999937,0.000042,-0.000182,0.000011,9471.527448,0.999978,USDC/USD,1.598389e-06,avg_return,True
513680,2024-09-25 23:51:00+00:00,0.999937,0.000042,-0.000182,0.000011,9471.527448,0.999978,USDC/USD,1.598389e-06,avg_return,False
513680,2024-09-25 23:51:00+00:00,0.178291,-0.001240,0.003238,-0.000717,483.562375,1.003245,GRT/USD,1.517025e-10,return,True
513680,2024-09-25 23:51:00+00:00,0.178291,-0.001240,0.003238,-0.000717,483.562375,1.003245,GRT/USD,1.517025e-10,return,False
